In [1]:
include("experiments/experiments.jl")

┌ Info: Precompiling Plinf [3f517155-95b4-4010-b517-e038c017d66c]
└ @ Base loading.jl:1342
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



run_sips_inference2

In [2]:
print(EXPERIMENTS_PATH)

./experiments

In [4]:
run_domain_experiments(EXPERIMENTS_PATH, "taxi", "suboptimal", :sips)

Running experiments for problem 0...
Inferring goals for trajectory 0, goal 0...
True goal: and(passenger-at(green))
Running SIPS Reimplementation
n_goals 3
n_samples30
goal_strataDict((:goal_init => :goal) => [1, 2, 3])
start_time1.638255088567612e9
Timestep 1
Timestep 2
Timestep 3
Timestep 4
Timestep 5
Timestep 6
Timestep 7
Timestep 8
Timestep 9
Timestep 10
Timestep 11
Timestep 12
Timestep 13
Writing results to taxi_problem_0_goal0_0.csv...
Inferring goals for trajectory 1, goal 0...
True goal: and(passenger-at(green))
Running SIPS Reimplementation
n_goals 3
n_samples30
goal_strataDict((:goal_init => :goal) => [1, 2, 3])
start_time1.638255093365429e9
Timestep 1
Timestep 2
Timestep 3
Timestep 4
Timestep 5
Timestep 6
Timestep 7
Timestep 8
Timestep 9
Timestep 10
Timestep 11
Timestep 12
Timestep 13
Writing results to taxi_problem_0_goal0_1.csv...
Inferring goals for trajectory 2, goal 1...
True goal: and(passenger-at(yellow))
Running SIPS Reimplementation
n_goals 3
n_samples30
goal_strat

Inferring goals for trajectory 0, goal 0...
True goal: and(passenger-at(red))
Running SIPS Reimplementation
n_goals 3
n_samples30
goal_strataDict((:goal_init => :goal) => [1, 2, 3])
start_time1.638255236835276e9
Timestep 1
Timestep 2
Timestep 3
Timestep 4
Timestep 5
Timestep 6
Timestep 7
Timestep 8
Timestep 9
Timestep 10
Timestep 11
Timestep 12
Timestep 13
Timestep 14
Timestep 15
Writing results to taxi_problem_3_goal0_0.csv...
Inferring goals for trajectory 1, goal 0...
True goal: and(passenger-at(red))
Running SIPS Reimplementation
n_goals 3
n_samples30
goal_strataDict((:goal_init => :goal) => [1, 2, 3])
start_time1.638255245936004e9
Timestep 1
Timestep 2
Timestep 3
Timestep 4
Timestep 5
Timestep 6
Timestep 7
Timestep 8
Timestep 9
Timestep 10
Timestep 11
Timestep 12
Timestep 13
Timestep 14
Timestep 15
Timestep 16
Timestep 17
Timestep 18
Timestep 19
Writing results to taxi_problem_3_goal0_1.csv...
Inferring goals for trajectory 2, goal 1...
True goal: and(passenger-at(green))
Running 

(24×16 DataFrame
 Row │ q1_true_goal_prob  mid_true_goal_prob  q3_true_goal_prob  end_true_goal ⋯
     │ Float64            Float64             Float64            Float64       ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │       0.333333            0.666667           0.666667               1.0 ⋯
   2 │       0.444444            0.0                0.0                    0.0
   3 │       0.285714            0.571428           0.762112               1.0
   4 │       0.277778            0.444444           0.571429               1.0
   5 │       1.68147e-6          1.7584e-19         1.0                    1.0 ⋯
   6 │       0.285714            0.500416           0.647439               1.0
   7 │       0.238095            0.0                0.0                    0.0
   8 │       0.571428            1.0                0.999741               1.0
   9 │       0.357143            0.375              0.500017               1.0 ⋯
  10 │       3.41745e-3

In [ ]:
function world_particle_filter2(
    world_init::WorldInit,
    world_config::WorldConfig,
    observed_trajectory::Vector{State},
    observed_terms::Vector{<:Term},
    num__particles::Int;
    strata=nothing,
    callback=nothing,
    resample=true,
    rejuvenate=nothing)
    
    threshold::Float64=1/4
    @unpack domain = world_config
    num_observations = length(observed_trajectory) 
    time_steps = collect(1:num_observations)
    observed_traj_choices = traj_choicemaps(observed_trajectory, 
                                            domain, 
                                            observed_terms)
    
    world_args = (world_init, world_config)
    argdiffs = (UnknownChange(), NoChange(), NoChange()) # Gen types, 
                                                         # UknownChannge: no information of change is provided
                                                         # NoChange: value is unchanged
    
    # if strata=nothing then this simply calls initialize_particle_filter
    beleif_state =  initialize_pf_stratified(world_model,    # world model
                                         (0, world_args...), # the initial arugments for the world model
                                         choicemap(),        # initial observations expressed as choicemap
                                         strata,             # goal_strata
                                         num_particles)        # number of particles
    
    # iterate through each timestep
    for (time_step, t) in enumerate(time_steps)
        if resample && get_ess(beleif_state) < (num_particles * threshold)
            @debug "Resampling..."
            pf_residual_resample!(beleif_state, priority_fn=w->w*0.75)
            if rejuvenate != nothing 
                rejuvenate(beleif_state) 
            end
        end
        
        t_prev = t-1
        if time_step == 1
            t_prev = 0
        end
            
        pf_update!(beleif_state, 
                   (t, world_args...), 
                   argdiffs, 
                   observed_traj_choices[time_step])
          
        if callback != nothing 
            traces_t = get_traces(beleif_state)
            weights_t = get_log_norm_weights(beleif_state)
            callback(t, observed_trajectory[t], traces_t, weights_t)
        end
    
    end
        
    traces = get_traces(beleif_state)
    weights = get_log_norm_weights(beleif_state)
    lml_est = logsumexp(get_log_weights(beleif_state)) - log(num_particles)    
    return traces, weights, lml_est
end


In [ ]:
priority_fn=w->w*0.75

In [ ]:
priority_fn(1)

In [ ]:
priority_fn(3)

In [ ]:
3*.75